# LeRobotを使った模倣学習
このノートブックでは，ロボットラーニングのためのオープンソースプロジェクトである[LeRobot](https://github.com/huggingface/lerobot/tree/main)を用いて，模倣学習を実際に回し，モデルの性能を評価します．

## 環境構築

In [1]:
import lerobot
import torch
import sys

print("using", sys.executable)
print("LeRobot", getattr(lerobot, "__version__", "dev"), "CUDA", torch.cuda.is_available())

using /home/ninja/pai_training/.pixi/envs/default/bin/python
LeRobot 0.1.0 CUDA True


## 学習の実行
Huggingface Hub上にある `lerobot/pusht` というデータセットをダウンロードして，拡散モデルをベースにした模倣学習アルゴリズムであるDiffusion Policyを学習します．
以下のリンクから，web上でHugging Face Hubに上がっている様々なデータセットを可視化することができます．

https://huggingface.co/spaces/lerobot/visualize_dataset

また，データセットの詳細も公開されています．

`lerobot/pusht`の詳細：https://huggingface.co/datasets/lerobot/pusht
```
{
    "codebase_version": "v2.0",
    "robot_type": "unknown",
    "total_episodes": 206,
    "total_frames": 25650,
    "total_tasks": 1,
    "total_videos": 206,
    "total_chunks": 1,
    "chunks_size": 1000,
    "fps": 10,
    ...
}
```

In [4]:
!cd ~/pai_training/lerobot

In [5]:
!pwd

/home/ninja/pai_training


In [7]:
!find lerobot -name train.py


lerobot/lerobot/scripts/train.py
lerobot/lerobot/configs/train.py


In [8]:
! pixi run python -m lerobot.scripts.train \
  --dataset.repo_id='lerobot/pusht' \
  --policy.type=diffusion \
  --steps=3000 \
  --batch_size=32 \
  --output_dir=outputs/train/lerobot_pusht \
  --job_name=lerobot_pusht \
  --policy.device=cuda \
  --wandb.enable=false

INFO 2025-07-31 10:22:14 ts/train.py:111 {'batch_size': 32,                                       
 'dataset': {'episodes': None,
             'image_transforms': {'enable': False,
                                  'max_num_transforms': 3,
                                  'random_order': False,
                                  'tfs': {'brightness': {'kwargs': {'brightness': [0.8,
                                                                                   1.2]},
                                                         'type': 'ColorJitter',
                                                         'weight': 1.0},
                                          'contrast': {'kwargs': {'contrast': [0.8,
                                                                               1.2]},
                                                       'type': 'ColorJitter',
                                                       'weight': 1.0},
                                          'hue': {'kw

In [10]:
!ls ./outputs/train/lerobot_pusht/checkpoints/003000/pretrained_model

config.json  model.safetensors	train_config.json


## モデル評価
上で学習されたモデルを評価する．

In [12]:
! pixi run python -m lerobot.scripts.eval \
    --policy.path=outputs/train/lerobot_pusht/checkpoints/003000/pretrained_model \
    --env.type=pusht \
    --eval.batch_size=10 \
    --eval.n_episodes=10 \
    --policy.use_amp=false \
    --policy.device=cuda \
    --vis.rerun=true

usage: eval.py [-h] [--config_path str] [--env str]                                               
               [--env.type {aloha,pusht,xarm}] [--env.task str]
               [--env.fps str] [--env.features str] [--env.features_map str]
               [--env.episode_length str] [--env.obs_type str]
               [--env.render_mode str] [--env.visualization_width str]
               [--env.visualization_height str] [--eval str]
               [--eval.n_episodes str] [--eval.batch_size str]
               [--eval.use_async_envs str] [--policy str]
               [--policy.type {act,diffusion,pi0,tdmpc,vqbet,pi0fast}]
               [--policy.replace_final_stride_with_dilation str]
               [--policy.pre_norm str] [--policy.dim_model str]
               [--policy.n_heads str] [--policy.dim_feedforward str]
               [--policy.feedforward_activation str]
               [--policy.n_encoder_layers str] [--policy.n_decoder_layers str]
               [--policy.use_vae str] [--po